In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.utils import get_acc_df
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
df100, df200 = pd.DataFrame(), pd.DataFrame()
for i in range(1,15):
    session = fastf1.get_session(2025,i,'Race')
    session.load(laps=True,telemetry=True,messages=False,weather=False)
    if i == 13:
        continue
    acc_df = get_acc_df(session)
    df100 = pd.concat([df100,acc_df['0-100'].rename(str(i))],axis=1)
    df200 = pd.concat([df200,acc_df['100-200'].rename(str(i))],axis=1)

In [ ]:
start_df = df100.T.copy()

In [ ]:
start_df.loc[['7','9','14'],'TSU'] = np.nan
start_df.loc[['1','2','10'],'LAW'] = np.nan
start_df.loc[['1','12'],'BEA'] = np.nan
start_df.loc[['6','10'],'GAS'] = np.nan
start_df.loc['12','RUS'] = np.nan
start_df.loc['12','LEC'] = np.nan
start_df.loc['12','BOR'] = np.nan
start_df.loc['12','HAD'] = np.nan

In [ ]:
rel_start_df = round(start_df.div(start_df.min(axis=1),axis=0),3)

In [ ]:
order = rel_start_df.median(axis=0).sort_values().index

In [ ]:
dcm = plotting.get_driver_color_mapping(session)
dcm['DOO'] = '#ff87bc'

In [ ]:
setup_plot(xyticksize=20,axeslabel=20)

fig, ax = plt.subplots(figsize=(15,15))

sns.boxplot(data=rel_start_df, orient='h', order=order, fliersize=0, palette=dcm,
            linecolor='w', boxprops=dict(alpha=0.2),ax=ax)
sns.stripplot(data=rel_start_df, orient='h', order=order, size=10, palette=dcm, ax=ax)

ax.set_xlim(left=0.99,right=1.35)
ax.set_xlabel('Gap (%)')
ax.set_xticks([i/100 for i in range(100,140,5)])
ax.set_xticklabels(['Fastest']+[str(i) for i in range(5,40,5)])
ax.set_title('Race Start 0-100 Times (Relative)')
ax.axvline(1.0, color='white', linestyle='--', linewidth=1)
ax.grid(axis='x', linestyle='--')

In [ ]:
field_avg = rel_start_df.median().mean()
field_avg

In [ ]:
mean_dev = (rel_start_df.median() - field_avg).sort_values()*100
mean_dev

In [ ]:
setup_plot(xyticksize=24,figtitle=30)

# Define the reference point of fastest driver
fastest_line = round((1 - field_avg)*100,2)  # percent relative to field avg

fig, ax = plt.subplots(figsize=(16, 18))
bars = plt.barh(mean_dev.index, mean_dev.values, color=[dcm[driver] for driver in mean_dev.index])

# Reference lines
ax.axvline(x=0, color='w', linewidth=4, linestyle='-')  # Field avg
# ax.axvline(x=fastest_line, color='w', linestyle='--', linewidth=2)
# ax.text(fastest_line, -1.5, "Fastest\nStart", color='w', ha='center', fontsize=20)

# Add value labels
for driver, value in mean_dev.items():
    color = dcm[driver]
    ax.text(
        value + (0.2 if value >= 0 else -0.2),
        mean_dev.index.get_loc(driver),
        f"{value:+.2f}%",
        va='center',
        ha='left' if value >= 0 else 'right',
        fontsize=20,
        fontweight='bold',
        color=color
    )

# Titles and labels
ax.set_xlabel("")
# ax.set_title("Race Start Performance (-ve = Faster) wrt to Field Avg",pad=20)
ax.set_title("")
ax.set_xlim(-7,7)
ax.set_xticks([])
ax.invert_yaxis()
ax.set_ymargin(0.01)
ax.grid(axis='x', linestyle='--', alpha=0.3)

plt.show()


In [ ]:
fig.savefig('./media/Post4/race_start.png',dpi=300, bbox_inches='tight')

In [ ]:
best_starts_df = (rel_start_df == 1).sum(axis=0)
best_starts_df[best_starts_df > 0].sort_values(ascending=False)